In [17]:
import os


import pandas as pd
import numpy as np
import re

from collections import defaultdict
from collections import Counter

from sqlalchemy import create_engine
import sqlite3
import pickle


## Build Database

In [18]:
dataset=pd.read_csv('courses_with_keywords.csv')

In [19]:
dataset.head(2)

,title,url,categories,description_short,description_long,difficulty,duration,free_option,number_of_enroll,rating,paid_option,language,subtitle,platform,provider,image_url,popularity_index,keywords
0,Getting Started with AWS Mainframe Modernizati...,https://explore.skillbuilder.aws/learn/course/...,"Computer Science , Cloud , AWS ,Cloud Computing",NaN,NaN,0,0,1,0.0,5.0,0,English,English,0,AWS Skill Builder,https://ccweb.imgix.net/https%3A%2F%2Fwww.clas...,0.415333,"getting, start, aws, mainframe, modernization,..."
1,Cloud for CEOs,https://explore.skillbuilder.aws/learn/course/...,"Computer Science , Cloud , AWS ,Cloud Computing",This course provides CEOs and presidents a hig...,This course provides CEOs and presidents a hig...,0,0,1,0.0,5.0,0,English,English,0,AWS Skill Builder,https://ccweb.imgix.net/https%3A%2F%2Fwww.clas...,0.415333,"cloud, ceo, course, provide, ceo, president, h..."


In [20]:
dataset.isna().sum()

title                  0
url                    0
categories             0
description_short    216
description_long     118
difficulty             0
duration               0
free_option            0
number_of_enroll       0
rating                 0
paid_option           70
language               0
subtitle               0
platform               0
provider               0
image_url              0
popularity_index       0
keywords               0
dtype: int64

In [21]:
dataset[dataset.isnull().any(axis=1)]

,title,url,categories,description_short,description_long,difficulty,duration,free_option,number_of_enroll,rating,paid_option,language,subtitle,platform,provider,image_url,popularity_index,keywords
0,Getting Started with AWS Mainframe Modernizati...,https://explore.skillbuilder.aws/learn/course/...,"Computer Science , Cloud , AWS ,Cloud Computing",NaN,NaN,0,0,1,0.0,5.000000,0,English,English,0,AWS Skill Builder,https://ccweb.imgix.net/https%3A%2F%2Fwww.clas...,0.415333,"getting, start, aws, mainframe, modernization,..."
8,Getting Started with AWS Cloud Essentials,https://explore.skillbuilder.aws/learn/course/...,"Computer Science , Cloud , AWS ,Cloud Computing",NaN,NaN,0,0,1,0.0,5.000000,0,English,English,0,AWS Skill Builder,https://ccweb.imgix.net/https%3A%2F%2Fwww.clas...,0.415333,"getting, start, aws, cloud, essentials, comput..."
12,Getting Started with AWS IoT SiteWise,https://explore.skillbuilder.aws/learn/course/...,"Computer Science , Cloud , AWS ,Cloud Computing",NaN,NaN,0,0,1,0.0,5.000000,0,English,English,0,AWS Skill Builder,https://ccweb.imgix.net/https%3A%2F%2Fwww.clas...,0.415333,"getting, start, aws, iot, sitewise, computer, ..."
13,Getting Started with AWS Control Tower,https://explore.skillbuilder.aws/learn/course/...,"Computer Science , Cloud , AWS ,Cloud Computing",NaN,NaN,0,0,1,0.0,5.000000,0,English,English,0,AWS Skill Builder,https://ccweb.imgix.net/https%3A%2F%2Fwww.clas...,0.415333,"getting, start, aws, control, tower, computer,..."
25,Getting Started with Amazon Timestream,https://explore.skillbuilder.aws/learn/course/...,"Computer Science , Cloud , AWS ,Cloud Computing",NaN,NaN,0,0,1,0.0,5.000000,0,English,English,0,AWS Skill Builder,https://ccweb.imgix.net/https%3A%2F%2Fwww.clas...,0.415333,"getting, start, amazon, timestream, computer, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19827,Systems and Application Security,https://www.coursera.org/learn/systems-applica...,"Information Technology, Security",Welcome to Systems and Application Security Co...,Welcome to Systems and Application Security Co...,0,1,1,2068.0,4.800000,NaN,English,English,12,(ISC)Â²,https://about.coursera.org/static/blueCoursera...,0.510040,"systems, application, security, welcome, syste..."
19828,TSI Math Prep MOOC,https://www.coursera.org/learn/tsi-math-prep,"Math and Logic, Math and Logic",The purpose of this course is to review and pr...,The purpose of this course is to review and pr...,0,1,1,0.0,3.976219,NaN,English,English,12,University of North Texas,https://about.coursera.org/static/blueCoursera...,0.649941,"tsi, math, prep, mooc, purpose, course, review..."
19845,Upgrading and Monitoring the Apigee Hybrid API...,https://www.coursera.org/learn/upgrading-monit...,"Information Technology, Cloud Computing",This course discusses the upgrade process for ...,This course discusses the upgrade process for ...,0,0,1,0.0,3.976219,NaN,English,English,12,Google Cloud,https://about.coursera.org/static/blueCoursera...,0.649941,"upgrading, monitor, apigee, hybrid, api, platf..."
19847,Using Asylo to Protect Secret Data from an Att...,https://www.coursera.org/learn/asylo-protect-s...,"Information Technology, Security","In this Google Cloud Lab, you build a simple e...","In this Google Cloud Lab, you build a simple e...",1,0,1,0.0,3.976219,NaN,English,English,12,Google Cloud,https://about.coursera.org/static/blueCoursera...,0.649941,"asylo, protect, secret, data, attacker, root, ..."


In [22]:
dataset['description_short'] = dataset['description_short'].apply(lambda x: 'Description not availabel' if pd.isna(x) else x)
dataset['description_long'] = dataset['description_long'].apply(lambda x: 'Description not availabel' if pd.isna(x) else x)
dataset['paid_option'] =dataset['paid_option'].apply(lambda x: 'Refer link' if pd.isna(x) else x)

In [23]:
# Load Course Data from pickle
pkl_file = 'course_data.pickle'
dataset_db = pd.read_pickle(pkl_file)
dataset_header = ['index']+ list(dataset_db.columns)
dataset_db['index'] = dataset_db.index + 1
dataset_db = dataset_db[dataset_header]


course_header = ['courseID',
                 'title',
                 'url',
                 'categories',
                 'description_short',
                 'description_long',
                 'difficulty',
                 'duration',
                 'free_option',
                 'number_of_enroll',
                 'rating',
                 'paid_option',
                 'language',
                 'subtitle',
                 'platform',
                 'provider',
                 'image_url',
                 'popularity_index',
                 'keywords']

dataset_db.columns = course_header
dataset_db.head()

,courseID,title,url,categories,description_short,description_long,difficulty,duration,free_option,number_of_enroll,rating,paid_option,language,subtitle,platform,provider,image_url,popularity_index,keywords
0,1,Getting Started with AWS Mainframe Modernizati...,https://explore.skillbuilder.aws/learn/course/...,"Computer Science , Cloud , AWS ,Cloud Computing",Description not availabel,Description not availabel,0,0,1,0.0,5.0,0,English,English,0,AWS Skill Builder,https://ccweb.imgix.net/https%3A%2F%2Fwww.clas...,0.415333,"getting, start, aws, mainframe, modernization,..."
1,2,Cloud for CEOs,https://explore.skillbuilder.aws/learn/course/...,"Computer Science , Cloud , AWS ,Cloud Computing",This course provides CEOs and presidents a hig...,This course provides CEOs and presidents a hig...,0,0,1,0.0,5.0,0,English,English,0,AWS Skill Builder,https://ccweb.imgix.net/https%3A%2F%2Fwww.clas...,0.415333,"cloud, ceo, course, provide, ceo, president, h..."
2,3,Getting Started with AWS Mainframe Modernizati...,https://explore.skillbuilder.aws/learn/course/...,"Computer Science , Cloud , AWS ,Cloud Computing",The AWS Mainframe Modernization service helps ...,The AWS Mainframe Modernization service helps ...,0,0,1,0.0,5.0,0,English,English,0,AWS Skill Builder,https://ccweb.imgix.net/https%3A%2F%2Fwww.clas...,0.415333,"getting, start, aws, mainframe, modernization,..."
3,4,Introduction to Robotics on AWS,https://explore.skillbuilder.aws/learn/course/...,"Computer Science , Cloud , AWS ,Cloud Computing",The robotics industry is growing at a rapid ra...,The robotics industry is growing at a rapid ra...,0,0,1,0.0,5.0,0,English,English,0,AWS Skill Builder,https://ccweb.imgix.net/https%3A%2F%2Fwww.clas...,0.415333,"introduction, robotic, aw, robotic, industry, ..."
4,5,Getting Started with Bottlerocket,https://explore.skillbuilder.aws/learn/course/...,"Computer Science , Cloud , AWS ,Cloud Computing","Bottlerocket is a Linux-based, open-source ope...","Bottlerocket is a Linux-based, open-source ope...",0,0,1,0.0,5.0,0,English,English,0,AWS Skill Builder,https://ccweb.imgix.net/https%3A%2F%2Fwww.clas...,0.415333,"getting, start, bottlerocket, bottlerocket, li..."


In [24]:
# SQL QUERY - CREATE TABLE

course_create_table = ''' CREATE TABLE IF NOT EXISTS course 
                        (
                        courseID INTEGER NOT NULL PRIMARY KEY,
                        title TEXT NOT NULL,
                        url TEXT NOT NULL,
                        categories TEXT,
                        description_short TEXT,
                        description_long TEXT,
                        difficulty INTEGER NOT NULL,
                        duration INTEGER NOT NULL,
                        free_option INTEGER,
                        number_of_enroll INTEGER,
                        rating REAL,
                        paid_option TEXT,
                        language TEXT,
                        subtitle TEXT,
                        platform INTEGER NOT NULL,
                        provider TEXT,
                        image_url TEXT, 
                        popularity_index REAL,
                        keywords TEXT
                        );
                        '''



user_create_table = ''' CREATE TABLE IF NOT EXISTS user
                        (
                        userID INTEGER NOT NULL PRIMARY KEY, 
                        name varchar(255) NOT NULL,
                        username varchar(255) NOT NULL UNIQUE, 
                        password varchar(255) NOT NULL,
                        email varchar(255) NOT NULL
                        );
                        '''

favourite_create_table  = ''' CREATE TABLE IF NOT EXISTS favourite
                             (
                             userID INTEGER NOT NULL,
                             courseID INTEGER NOT NULL,
                             FOREIGN KEY (userID) REFERENCES user (userID),
                             FOREIGN KEY (courseID) REFERENCES course (courseID)
                             );
                             '''


query_create_table = ''' CREATE TABLE IF NOT EXISTS query
                         (
                         queryID INTEGER PRIMARY KEY, 
                         query_count INTEGER NOT NULL, 
                         userID INTEGER NOT NULL,
                         query_time DATETIME DEFAULT (CURRENT_TIMESTAMP),
                         query_text TEXT, 
                         query_duration INTEGER,
                         query_difficulty INTEGER, 
                         query_free_option INTEGER,
                         FOREIGN KEY (userID) REFERENCES user (userID)
                         );
                         '''

recom_create_table = ''' CREATE TABLE IF NOT EXISTS recommendation
                         (
                         userID INTEGER NOT NULL, 
                         query_count INTEGER NOT NULL, 
                         ranking INTEGER NOT NULL, 
                         courseID INTEGER NOT NULL,
                         FOREIGN KEY (userID) REFERENCES user (userID),
                         FOREIGN KEY (query_count) REFERENCES query (query_count),
                         FOREIGN KEY (courseID) REFERENCES course (courseID)
                         );

                         '''



# skill_create_table  = ''' CREATE TABLE IF NOT EXISTS skill
#                              (
#                              userID INTEGER NOT NULL,
#                              skill TEXT,
#                              FOREIGN KEY (userID) REFERENCES user (userID)
                            
#                              );
#                              '''
career_path_create_table ='''   
                            CREATE TABLE IF NOT EXISTS career_path
                            (
                            userID INTEGER NOT NULL,
                            career TEXT,
                            FOREIGN KEY (userID) REFERENCES user (userID)
                            );
                            '''
complete_course_path_create_table='''
                                     CREATE TABLE IF NOT EXISTS completedCourses
                             (
                             userID INTEGER NOT NULL,
                             courseID INTEGER NOT NULL,
                             rating INTEGER,
                             skill TEXT,
                             FOREIGN KEY (userID) REFERENCES user (userID),
                             FOREIGN KEY (courseID) REFERENCES course (courseID)
                             );
                             '''

                                

In [25]:
# EXECUTE CREATE TABLE

filename = 'app_database.db'

conn = sqlite3.connect(filename)
conn.execute(course_create_table)
conn.execute(user_create_table)
conn.execute(favourite_create_table)
conn.execute(query_create_table)
conn.execute(recom_create_table)
# conn.execute(skill_create_table)
conn.execute(career_path_create_table)
conn.execute(complete_course_path_create_table)

conn.close()

In [26]:
# SQL QUERY - INSERT DATA

insert_course = ''' INSERT OR REPLACE INTO course 
                        (
                        courseID,
                        title,
                        url,
                        categories,
                        description_short,
                        description_long,
                        difficulty,
                        duration,
                        free_option,
                        number_of_enroll,
                        rating,
                        paid_option,
                        language,
                        subtitle,
                        platform,
                        provider,
                        image_url,
                        popularity_index,
                        keywords
                        )
                        VALUES
                        (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
                        '''

In [27]:
conn.close()

In [28]:
# EXECUTE INSERT DATA

filename = 'app_database.db'

conn = sqlite3.connect(filename)
dataset_db.to_sql(name='course', con=conn, if_exists='append', index=False)
conn.close()

In [29]:
#Reload Database for Checking
# REQUERY FOR CHECKING

filename = 'app_database.db'
course_table = 'course'

sqlite_conn = sqlite3.connect(filename)
course_check = pd.read_sql('SELECT * FROM ' + course_table, sqlite_conn)
sqlite_conn.close()

In [30]:
course_check.head()

,courseID,title,url,categories,description_short,description_long,difficulty,duration,free_option,number_of_enroll,rating,paid_option,language,subtitle,platform,provider,image_url,popularity_index,keywords
0,1,Getting Started with AWS Mainframe Modernizati...,https://explore.skillbuilder.aws/learn/course/...,"Computer Science , Cloud , AWS ,Cloud Computing",Description not availabel,Description not availabel,0,0,1,0,5.0,0,English,English,0,AWS Skill Builder,https://ccweb.imgix.net/https%3A%2F%2Fwww.clas...,0.415333,"getting, start, aws, mainframe, modernization,..."
1,2,Cloud for CEOs,https://explore.skillbuilder.aws/learn/course/...,"Computer Science , Cloud , AWS ,Cloud Computing",This course provides CEOs and presidents a hig...,This course provides CEOs and presidents a hig...,0,0,1,0,5.0,0,English,English,0,AWS Skill Builder,https://ccweb.imgix.net/https%3A%2F%2Fwww.clas...,0.415333,"cloud, ceo, course, provide, ceo, president, h..."
2,3,Getting Started with AWS Mainframe Modernizati...,https://explore.skillbuilder.aws/learn/course/...,"Computer Science , Cloud , AWS ,Cloud Computing",The AWS Mainframe Modernization service helps ...,The AWS Mainframe Modernization service helps ...,0,0,1,0,5.0,0,English,English,0,AWS Skill Builder,https://ccweb.imgix.net/https%3A%2F%2Fwww.clas...,0.415333,"getting, start, aws, mainframe, modernization,..."
3,4,Introduction to Robotics on AWS,https://explore.skillbuilder.aws/learn/course/...,"Computer Science , Cloud , AWS ,Cloud Computing",The robotics industry is growing at a rapid ra...,The robotics industry is growing at a rapid ra...,0,0,1,0,5.0,0,English,English,0,AWS Skill Builder,https://ccweb.imgix.net/https%3A%2F%2Fwww.clas...,0.415333,"introduction, robotic, aw, robotic, industry, ..."
4,5,Getting Started with Bottlerocket,https://explore.skillbuilder.aws/learn/course/...,"Computer Science , Cloud , AWS ,Cloud Computing","Bottlerocket is a Linux-based, open-source ope...","Bottlerocket is a Linux-based, open-source ope...",0,0,1,0,5.0,0,English,English,0,AWS Skill Builder,https://ccweb.imgix.net/https%3A%2F%2Fwww.clas...,0.415333,"getting, start, bottlerocket, bottlerocket, li..."


In [31]:
print(course_check.shape)

(20475, 19)


In [32]:
course_check.isna().sum()

courseID             0
title                0
url                  0
categories           0
description_short    0
description_long     0
difficulty           0
duration             0
free_option          0
number_of_enroll     0
rating               0
paid_option          0
language             0
subtitle             0
platform             0
provider             0
image_url            0
popularity_index     0
keywords             0
dtype: int64